<a href="https://colab.research.google.com/github/andrewcarr6239/portfolio/blob/gh-pages/ExcelReportGeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openpyxl google_spreadsheet google-auth-oauthlib gsheets gspread
!pip install — upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth', 400)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import os
from datetime import date, timedelta, datetime 
import io
import smtplib
from email.mime.application import MIMEApplication
import email, smtplib, ssl
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from datetime import datetime
from os import path, remove
import configparser
import pandas as pd
import shutil
import openpyxl 
import openpyxl 
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Alignment
from openpyxl.worksheet.table import Table, TableStyleInfo
from openpyxl.styles import Font, Fill
from openpyxl.styles.colors import Color
import numpy as np
import datetime
from gsheets import Sheets
import gspread
from oauth2client.client import GoogleCredentials
import time

In [ ]:
from google.colab import auth
auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())

## Import Sheets from GDrive with Gsheets

In [ ]:
def openGsheet(key):
  sheet = gc.open_by_key(key)
  worksheet = sheet.get_worksheet(0)
  dataframe = pd.DataFrame(worksheet.get_all_records())
  return dataframe

### Import Form output CSV and select previous week

In [ ]:
def importpdForm(pdDataFrame, subject):
  # ImportformCSV which returns pd dataframe with reduced columns relevant to teachers
  df = pdDataFrame
  

  # Select Specific Subject
  nan_value = float("NaN")
  df.replace("", nan_value, inplace=True)
  dropped = df.dropna(subset=[subject, 'Student notes'])
  dropped['Timestamp'] = pd.to_datetime(dropped['Timestamp'])
  dropped['Student notes'] = dropped['Student Name'] + ': ' + dropped['Student notes']
  # dropped = dropped[dropped.Timestamp.between((pd.Timestamp('now') - pd.Timedelta(18, unit='D')),pd.Timestamp('today').floor('D'))]
  dropped1 = dropped.loc['2021-5-27':'2021-6-4']
  dropped = dropped.set_index('Timestamp') 
  dropped = dropped.sort_index()

  # Select last week only
  maths = dropped[['Student notes']]
  
  # Select only subset of columns

  # Return pandas dataframe object
  return maths

### Convert df to styled excel spreadsheet

In [ ]:
def stylePandasSavetoxlsx(pddf,saveLocation):
  # Convert pandas df to csv with alternating colour excel and 
    c = Color(indexed=32)
    c = Color(theme=6, tint=0.5)

    # import Font function from openpyxl 
    from openpyxl.styles import Font 

    wb = openpyxl.Workbook() 
    sheet = wb.active 

    for r in dataframe_to_rows(pddf, index=True, header=True):
        sheet.append(r)

    tab = Table(displayName="Table1", ref="A1:M100")

    style = TableStyleInfo(name="TableStyleMedium6", showFirstColumn=False,
                          showLastColumn=False, showRowStripes=False, showColumnStripes=True)
    tab.tableStyleInfo = style

    '''
    Table must be added using ws.add_table() method to avoid duplicate names.
    Using this method ensures table name is unque through out defined names and all other table name. 
    '''
    sheet.add_table(tab)

    sheet.title = "New Title"

    # row = sheet.row_dimensions[1]
    # col = ws.column_dimensions['A']

    c = sheet['A1']

    for row in sheet.iter_rows():
        for cell in row:      
            cell.alignment =  cell.alignment.copy(wrapText=True)

    sheet.sheet_properties.tabColor = "1072BA"
    sheet.column_dimensions['A'].width = 20
    sheet.column_dimensions['B'].width = 140
    sheet.column_dimensions['C'].width = 100
    sheet.column_dimensions['D'].width = 100
    sheet.column_dimensions['E'].width = 25
    sheet.column_dimensions['F'].width = 25
    sheet.column_dimensions['G'].width = 25
    sheet.column_dimensions['I'].width = 40
    sheet.column_dimensions['J'].width = 40
    sheet.column_dimensions['L'].width = 20
    sheet.column_dimensions['H'].width = 30
    sheet.column_dimensions['M'].width = 20



    c = sheet['C2']
    sheet.freeze_panes = c


    row = sheet.row_dimensions[2]
    row.font = Font(underline="single")
    row.font = Font(color="FF0000")


    wb.save(saveLocation) 


### Import Year, Subjects and File Location

In [ ]:
# Open nccdAnnotation excel file with location, sheet key etc
key = '1c-6OCNvV1pZoMm9HKjJ7_1KF6zp09A-iQkChzTstCVQ'

from google.colab import auth
auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())

yearSubjectFLocation = openGsheet(key)

In [ ]:
list = []
for i in range(len(yearSubjectFLocation)):

  # Get todays date and previous weeks date
  Previous_Date = str((datetime.datetime.today() - datetime.timedelta(days=18)).date())
  Current_Date = str(pd.Timestamp('today').date())

  # Define variables for each loop
  key = yearSubjectFLocation.iloc[i]['sheetKey'][:-1]
  subject = yearSubjectFLocation.iloc[i]['subject'][:-1]
  location = yearSubjectFLocation.iloc[i]['location'][1:-1] + Previous_Date + "_" + Current_Date + ".xlsx"
  print(location)

  # Run 
  sheet = openGsheet(key)

  # Import current sheet and select subject
  try:
    currentSheet = importpdForm(sheet,subject )
  

    # Style xlsx and save to location
    stylePandasSavetoxlsx(currentSheet,location)

  except:
    print ("Subject Selection Failed")


  list.append(location)


# Create Staff Email Test

In [ ]:
def teacherNames():
  # create table with teacher names, emails and relvant file end place
  subjects = ['Subject [English]', 'Subject [Maths]',
        'Subject [Social com]', 'Subject [Art]', 'Subject [Science]',
        'Subject [Sport and Rec]']
  dfSubTeach = pd.DataFrame(subjects)
  dfSubTeach['email'] = ['awalsh@arcadia.qld.edu.au','mgray@arcadia.qld.edu.au','sdunemann@arcadia.qld.edu.au','jstafford@arcadia.qld.edu.au','awalsh@arcadia.qld.edu.au','bhenson@arcadia.qld.edu.au']
  # dfSubTeach['fileLocation']=['']
  dfSubTeach.columns = ['Teacher','Email']

In [ ]:
def createStaffEmail():
  # Create email to send to teachers
  subject = "An email with attachment from Python"
  body = "This is an email with attachment sent from Python"
  sender_email = "acarr@arcadia.qld.edu.au"
  receiver_email = "andrew.carr.6239@gmail.com"
  password = "Crown1989!"
  cc = 'acarr@arcadia.qld.edu.au'

  # Create a multipart message and set headers
  message = MIMEMultipart()
  message["From"] = sender_email
  message["To"] = receiver_email
  message["Subject"] = subject
  message["CC"] = cc  # Recommended for mass emails

  # Add body to email
  message.attach(MIMEText(body, "plain"))

  filename = "/content/drive/MyDrive/testCSV.csv"  # In same directory as script

  # Open PDF file in binary mode
  with open(filename, "rb") as attachment:
      # Add file as application/octet-stream
      # Email client can usually download this automatically as attachment
      part = MIMEBase("application", "octet-stream")
      part.set_payload(attachment.read())

  # Encode file in ASCII characters to send by email    
  encoders.encode_base64(part)

  # Add header as key/value pair to attachment part
  part.add_header(
      "Content-Disposition",
      f"attachment; filename= {filename}",
  )

  # Add attachment to message and convert message to string
  message.attach(part)
  text = message.as_string()

  # Log in to server using secure context and send email
  context = ssl.create_default_context()
  with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
      server.login(sender_email, password)
      server.sendmail(sender_email, receiver_email, text)

In [ ]:
list = []

# Location of end excel files
fileLocation = "/content/drive/MyDrive/colabDevelopment/"

for i in range(len(yearSubjectFLocation[0:20])):

  # Define variables for each loop
  key = yearSubjectFLocation.iloc[i]['sheetKey'][:-1]
  year = yearSubjectFLocation.iloc[i]['year']
  # location = yearSubjectFLocation.iloc[i]['location'][1:-1] + ".xlsx"

  # Run 
  sheet = openGsheet(key)

  # Determine list of subjects
  subjectList = [i for i in sheet.columns if "Subject" in i]

  for i in subjectList:
    # Define location
    location = fileLocation + str(year) + "/"  + str(i) + ".xlsx"

    # Import current sheet and select subject
    try:
      currentSheet = importpdForm(sheet, i)
    

      # Style xlsx and save to location
      stylePandasSavetoxlsx(currentSheet, location)

    except:
      print ("Subject Selection Failed")

    print(location)


  # list.append(location)

  # time.sleep(9)

print(list)